In [4]:
from bs4 import BeautifulSoup
import requests
#headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0'}
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:82.0) Gecko/20100101 Firefox/82.0'}

page = requests.get("https://www.loteriasyapuestas.es/es/resultados/primitiva", headers=headers)
soup = BeautifulSoup(page.content)
titulo = soup.title.string
print(titulo)
print(soup.prettify())


La Primitiva | Resultados OFICIALES - Loterías y Apuestas del Estado
<!DOCTYPE html>
<html>
 <head>
  <meta content="text/javascript" http-equiv="Content-Script-Type"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <script type="text/javascript">
   var timeToGetJsonpSaga = "180000";
	var hostReferrerSaga = "juegos.loteriasyapuestas.es";
  </script>
  <title>
   La Primitiva | Resultados OFICIALES - Loterías y Apuestas del Estado
  </title>
  <meta content="La Primitiva | Resultados OFICIALES - Loterías y Apuestas del Estado" name="keywords"/>
  <meta content="Revisa recaudación, botes, premios y ganadores de todos los sorteos de La Primitiva. Toda la información aquí" name="description"/>
  <meta content="2019 SELAE, Loterías y Apuestas del Estado" name="copyright"/>
  <meta content="La Primitiva | Resultados OFICIALES - Loterías y Apuestas del Estado" property="og:title"/>
  <meta con